In [75]:
import pyspark
from pyspark.sql import SparkSession

In [76]:
spark = SparkSession.builder.master("local[*]").appName("Datamanipulation").getOrCreate()

In [77]:
spark

In [78]:
# read our data - lives in a csv file

df = spark.read.option("header","true").csv("Sample - EU Superstore.csv")

In [79]:
# how many rows of the EU Superstore dataset have the country being France
df.filter(df["Country"]=="France").count()

2827

In [80]:
# of those, how many are profitable?
df.filter((df.Country == "France") & (df.Profit>0)).count()

2277

In [81]:
# how any different discount brackets exist? what are they?
df.select("Discount").distinct().show(15)

+--------+
|Discount|
+--------+
|     0.3|
|     0.7|
|       0|
|     0.2|
|    0.15|
|    0.35|
|     0.8|
|    0.45|
|     0.5|
|    0.65|
|     0.6|
|     0.1|
|    0.85|
|     0.4|
+--------+



In [82]:
# let's see the totl profit by discount bracket, make sure they are ordered by 


In [83]:
# what is the value after which we should stop offering discount?
df = df.withColumn("Real Price",df.Sales/(1-df.Discount))
df.select("Real Price", "Sales").show()

+------------------+--------+
|        Real Price|   Sales|
+------------------+--------+
|              79.2|    79.2|
|            388.92|  388.92|
|             35.19|   35.19|
|             50.94|   50.94|
|            307.44|  307.44|
|             122.4|   122.4|
|            413.82|  413.82|
|            428.22|  428.22|
|           3979.29| 3979.29|
|             43.56|   43.56|
|             25.26|   25.26|
|2715.4500000000003|2443.905|
|             12.21|   12.21|
|2549.7599999999998|2167.296|
|            153.45| 138.105|
|142.64999999999998| 128.385|
|            690.12|  690.12|
|              8.16|    8.16|
|            347.88|  347.88|
| 639.4499999999999| 575.505|
+------------------+--------+
only showing top 20 rows



In [84]:
# who are the top 5 most profitable customers
top5 = df.groupBy("Customer Name").agg({"Profit":"sum"}).orderBy("sum(Profit)",ascending=False).limit(5)


In [85]:
# get all the rows belonging to those 5 customer names: hint, you may need the collect method - how many rows are they?


In [86]:
# create a new column which is the value of the sale were there not discount applied. Hint: orginal = sales/(1-d)
df = df.withColumn("Real Price",df.Sales/(1-df.Discount))
df.select("Real Price", "Sales").show()

+------------------+--------+
|        Real Price|   Sales|
+------------------+--------+
|              79.2|    79.2|
|            388.92|  388.92|
|             35.19|   35.19|
|             50.94|   50.94|
|            307.44|  307.44|
|             122.4|   122.4|
|            413.82|  413.82|
|            428.22|  428.22|
|           3979.29| 3979.29|
|             43.56|   43.56|
|             25.26|   25.26|
|2715.4500000000003|2443.905|
|             12.21|   12.21|
|2549.7599999999998|2167.296|
|            153.45| 138.105|
|142.64999999999998| 128.385|
|            690.12|  690.12|
|              8.16|    8.16|
|            347.88|  347.88|
| 639.4499999999999| 575.505|
+------------------+--------+
only showing top 20 rows



In [87]:
# calculate the difference between sales and discount value
df.select(df["Real Price"]-df["Sales"]).show()

+--------------------+
|(Real Price - Sales)|
+--------------------+
|                 0.0|
|                 0.0|
|                 0.0|
|                 0.0|
|                 0.0|
|                 0.0|
|                 0.0|
|                 0.0|
|                 0.0|
|                 0.0|
|                 0.0|
|   271.5450000000001|
|                 0.0|
|  382.46399999999994|
|  15.344999999999999|
|  14.264999999999986|
|                 0.0|
|                 0.0|
|                 0.0|
|  63.944999999999936|
+--------------------+
only showing top 20 rows



In [88]:
# how much money did we not gain due to the discounts - per discount bracket?


In [89]:
# find the discount bracket which made us not gain the most (dynamically)



In [90]:
# what would have been the total profit if we removed all orders from that discount group? 


In [91]:
#how much more (or less) profit is that?



In [92]:
# create a temporary table for our superstore table in sql
df.createOrReplaceTempView("Superstore")

In [93]:
# use an SQL query to count the number of rows
rows = spark.sql("SELECT COUNT(*) FROM Superstore")
rows.show()

+--------+
|count(1)|
+--------+
|   10000|
+--------+



In [94]:
# Use an SQL query to calculate the profit ratio for each country: hint, ratio is sum(profit)/sum(sales)
profit_ratio = spark.sql("SELECT Country, SUM(Profit)/sum(Sales) FROM Superstore GROUP BY Country")
profit_ratio.show()


+--------------+--------------------------+
|       Country|(sum(Profit) / sum(Sales))|
+--------------+--------------------------+
|        Sweden|       -0.5745674280714466|
|       Germany|       0.17066792076621765|
|        France|       0.12693568221933804|
|       Belgium|       0.23508766583987942|
|       Finland|       0.18864296633316185|
|         Italy|       0.06844355185424991|
|        Norway|        0.2517747548521659|
|         Spain|       0.18941580658358978|
|       Denmark|       -0.4957190005664471|
|       Ireland|      -0.44426677493909256|
|   Switzerland|        0.2909201193350232|
|      Portugal|       -0.5761662270806188|
|       Austria|        0.2641908775042505|
|United Kingdom|       0.21170103540397134|
|   Netherlands|       -0.5298342790541865|
+--------------+--------------------------+



In [95]:
# is the country with the largest profit ratio, the country with the largest profit?

